In [1]:
from dotenv import load_dotenv

import openai
import os
load_dotenv()

import logging
import sys
import os

import numexpr as ne

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
openai.api_key = os.getenv('OPENAI_API_KEY')


os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

In [2]:
from llama_index import (
    GPTVectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    load_index_from_storage
)
from llama_index.prompts import PromptTemplate


try:
    storage_context = StorageContext.from_defaults(persist_dir='../storage/cache/andrew/medical_gpt_vector_store_index')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('../assets/AndrewHuberman/medical').load_data()
    index = GPTVectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir='../storage/cache/andrew/medical_gpt_vector_store_index/')
    print('persisting to disk')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.open

In [3]:
openai.log = "debug"

text_qa_template_str = (
    "Bạn là trợ lý của Andrew Huberman, người có thể đọc các ghi chú podcast của Andrew Huberman.\n"
    "Luôn trả lời truy vấn chỉ bằng cách sử dụng thông tin ngữ cảnh được cung cấp, "
    "chứ không phải kiến ​​thức sẵn có.\n"
    "Một số quy tắc cần tuân theo:\n"
    "1. Không bao giờ tham khảo trực tiếp bối cảnh nhất định trong câu trả lời của bạn.\n"
    "2. Tránh những câu như 'Dựa trên ngữ cảnh, ...' hoặc "
    "'Thông tin ngữ cảnh ...' hoặc bất cứ điều gì cùng"
    "những dòng đó.\n"
    "3. Phải trả lời câu truy vấn bằng thông tin ngữ cảnh đã được cung cấp chứ không phải kiến thức sẵn có của bạn.\n"
    "Thông tin bối cảnh dưới đây.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Trả lời câu hỏi:{query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [4]:
from llama_index.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="sentence-transformers/all-mpnet-base-v2", top_n=3
)

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from time import time

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[rerank],
    text_qa_template = text_qa_template
)

In [25]:
now = time()
response = query_engine.query(
    "Hậu môn bị sưng là bệnh gì?",
)
print(f"Elapsed: {round(time() - now, 2)}s")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Elapsed: 8.03s


In [26]:
print(response)

Hậu môn bị sưng là một tình trạng tổn thương vùng da ở lỗ hậu môn và ống dẫn hậu môn, thường do va chạm trong hoạt động tình dục quá mạnh hoặc sử dụng công cụ tình dục hỗ trợ. Tổn thương có thể gây ra chảy máu, vết xước, viêm nhiễm hoặc làm hậu môn bị sưng.


In [27]:
print(response.metadata)

{'927f3dad-dfb7-40b1-9f6c-ef89fc5c210d': {'file_path': '../assets/AndrewHuberman/medical/hau-mon-bi-sung', 'file_name': 'hau-mon-bi-sung', 'file_type': None, 'file_size': 21521, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, '52cad55f-a5f3-4ada-ae02-f8a7f5036ca9': {'file_path': '../assets/AndrewHuberman/medical/hau-mon-bi-sung', 'file_name': 'hau-mon-bi-sung', 'file_type': None, 'file_size': 21521, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, '7ded9810-761c-404b-9b7a-5630cb978272': {'file_path': '../assets/AndrewHuberman/medical/viem-hau-mon', 'file_name': 'viem-hau-mon', 'file_type': None, 'file_size': 23716, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}}


In [28]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='927f3dad-dfb7-40b1-9f6c-ef89fc5c210d', embedding=None, metadata={'file_path': '../assets/AndrewHuberman/medical/hau-mon-bi-sung', 'file_name': 'hau-mon-bi-sung', 'file_type': None, 'file_size': 21521, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c3f78ac8-80b1-41f4-86a1-7c15e5727191', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '../assets/AndrewHuberman/medical/hau-mon-bi-sung', 'file_name': 'hau-mon-bi-sung', 'file_type': None, 'file_size': 21521, 'creation_date': '2023-12-15', 'last_modified_date': '2023-12-15', 'last_accessed_date': '2023-12-15'}, ha

In [30]:
for i in response.source_nodes:
    print('score: ', i.score)
    print('text: ', i.text)
    print('='*100)

score:  0.5047247
text:  Những va chạm của hoạt động làm tình sẽ gây tổn thương vùng da ở lỗ hậu môn và ống dẫn hậu môn. Đặc biệt là khi quan hệ tình dục quá mạnh, hoặc sử dụng công cụ tình dục hỗ trợ, sự va chạm với vùng da hậu môn sẽ dễ gây ra những tổn thương như chảy máu, vết xước, viêm nhiễm hoặc làm hậu môn bị sưng. 
<h3><strong>8. Ung thư hậu môn</strong></h3> Ung thư hậu môn là tình trạng tế bào ung thư bắt đầu xuất hiện ở hậu môn. Dựa trên vị trí xuất hiện tế bào mà ung thư hậu môn được chia thành 2 nhóm gồm: Ung thư ống hậu môn và ung thư da quanh hậu môn. Ung thư biểu mô tế bào vảy ở hậu môn là loại ung thư hậu môn phổ biến nhất. Những nguyên nhân gây ra ung thư hậu môn thường là do:  Bệnh STD do nhiễm virus HPV Tuổi cao Rò hậu môn kéo dài Hậu môn thường xuyên gặp tình trạng kích thích   Ung thư hậu môn gây ra những triệu chứng như:  Hậu môn bị sưng hoặc nổi khối Chảy máu từ hậu môn Đau ống hậu môn Chảy máu hoặc tiết dịch    
<h2><strong>Chẩn đoán tình trạng hậu môn bị sưng 

# 2. Đánh giá RAG

In [ ]:
import pandas as pd

df = pd.read_csv('/Users/duongtrongchi/project/RAG/rag-app/test/test_case.csv')

df.head()

In [ ]:
from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index.embeddings import SimilarityMode


evaluator = SemanticSimilarityEvaluator(
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)

list_res = []
list_result = []
passed = []
for i in range(0, len(df)):
    reference = df['References'][i]
    response = query_engine.query(df['Questions'][i])

    result = await evaluator.aevaluate(
        response=response.response,
        reference=reference,
    )

    list_res.append(response.response)
    list_result.append(result.score)
    passed.append(result.passing)


In [ ]:
df['Result'] = list_result
df['Passed'] = passed
df['Responses'] = list_res

In [ ]:
df.to_csv('../results.csv', index=False)

In [ ]:
response = query_engine.query(
    "Đối tượng nào dễ bị xuất huyết bao tử?",
)


print(response)

In [ ]:
print(response.source_nodes)

In [ ]:
# Embedding Similarity Evaluator

from llama_index.evaluation import SemanticSimilarityEvaluator
from llama_index.embeddings import SimilarityMode


evaluator = SemanticSimilarityEvaluator(
    similarity_mode=SimilarityMode.DEFAULT,
    similarity_threshold=0.9,
)

reference = """
<h2><strong>Đối tượng nào dễ bị xuất huyết bao tử?</strong></h2> Xuất huyết dạ dày là tình trạng bệnh lý xảy ra phổ biến ở độ tuổi từ 20 &#8211; 50 tuổi, trong đó nam giới thường chiếm tỷ lệ cao hơn nữ giới bởi có xu hướng uống nhiều rượu bia hơn. Người lớn tuổi hay dùng các thuốc giảm đau chống viêm Non-steroid hoặc Aspirin cung có nguy cơ cao xuất huyết tiêu hóa.
"""

response = query_engine.query(
    "Bệnh bạch hầu là gì?",
)

result = await evaluator.aevaluate(
    response=response.response,
    reference=reference,
)

print("Score: ", result.score)
print("Passing: ", result.passing)